In [4]:
! pip install sentence_transformers

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.3/410.3 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.0 MB/s eta 0:00:00a 0:00:01
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
import os
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec


/opt/homebrew/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
with open('../secret.json') as f:
    data = json.load(f)
pinecone_key = data["pinecone_api"]
pc = Pinecone(api_key=pinecone_key)

# Check if the index exists, and if not, create a new one
index_name = 'codenet-python800'
if index_name not in pc.list_indexes().names():
    pc.create_index(index_name, dimension=384, metric='cosine', #dimension 384 for all-miniLM-L12-v2
            spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) )

# Connect to your Pinecone index
index = pc.Index(index_name)

In [15]:
def python_files_to_df():
    base_dir = '/Users/andrewyang/Desktop/workspace/turintech/LLM_BPD/Project_CodeNet_Python800'

    data = []
    index_number = 1

    # Iterate over each folder in the base directory
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                if file.endswith('.py'):
                    with open(file_path, 'r', encoding='utf-8') as file_content:
                        try:
                            code = file_content.read()
                            # Append the content along with the index to the data list
                            data.append({'index': index_number, 'codes': code})
                            index_number += 1
                        except Exception as e:
                            print(f"Error reading {file_path}: {e}")

    # Create a DataFrame from the data list
    df = pd.DataFrame(data, columns=['index', 'codes']).set_index('index')
    return df


In [16]:

# Load codeNet_python800
df = python_files_to_df()
print('data_prepared!')
print(df.head())

data_prepared!
                                                   codes
index                                                   
1      n=int(input())\na=list(map(int,input().split()...
2      s = int(input())\ns = input().rstrip().split('...
3      def lo2(n):\n    count = 0\n    while n%2==0:\...
4      a = int(input())\nb = list(map(int, input().sp...
5      N = int(input())\nA = list(map(int,input().spl...


In [17]:
# Initialize the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')  # Example model, replace with your choice

code_texts = df['codes'].tolist()


# Generate embeddings
embeddings = model.encode(code_texts, show_progress_bar=True).tolist()

Batches: 100%|██████████| 7500/7500 [28:56<00:00,  4.32it/s] 


In [26]:
df['embeddings'] = embeddings
# Prepare the data for upload
data_to_upload = list(zip(df.index, df['embeddings'], df['codes']))


In [31]:
len(data_to_upload)

240000

In [32]:
# Function to divide data into chunks for batch processing
def chunked_data(data, chunk_size):
    """Yield successive chunk_size chunks from data."""
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

# Upload data in batches
batch_size = 100
for batch in chunked_data(data_to_upload, batch_size):
    batch_to_upsert = [(str(id), vec, {'code': codes}) for id, vec, codes in batch]
    try:
        index.upsert(vectors=batch_to_upsert)
    except Exception as e:
        print('error, ', e)
        pass

error,  (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 22 Jul 2024 11:10:50 GMT', 'Content-Type': 'application/json', 'Content-Length': '116', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '17', 'x-pinecone-request-id': '7747599557177307485', 'x-envoy-upstream-service-time': '4', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata size is 203645 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}

error,  (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 22 Jul 2024 11:10:51 GMT', 'Content-Type': 'application/json', 'Content-Length': '116', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '20', 'x-pinecone-request-id': '4306047114874772378', 'x-envoy-upstream-service-time': '4', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata size is 203703 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}

error,  (400)
Reason: Bad Re